In [ ]:
!pip install pymongo

In [ ]:
## Import libraries
import os
import datetime
import pymongo
import pprint
import pandas as pd

In [ ]:
## Connect to the MONGO DB Instance

host_name = "localhost"
port = "27017"
atlas_cluster_name = "cluster0.0zywi.mongodb.net"
atlas_default_dbname = "test"
atlas_user_name = "ds2002"
atlas_password = "UVA!1819"

In [ ]:
conn_str = {
  #  "local" : f"mongodb://{host_name}:{port}/",
    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}/{atlas_default_dbname}"
}

client = pymongo.MongoClient(conn_str["atlas"])

#print(f"Local Connection String: {conn_str['local']}")
print(f"Atlas Connection String: {conn_str['atlas']}")

In [ ]:
client.list_database_names()
db = client['sample_restaurants']
restaurants1 = client.sample_restaurants
restaurants = db.restaurants

In [ ]:
## import and preview dataframe
from pandas.io.json import json_normalize
datapoints = list(db.restaurants.find({}))
df = json_normalize(datapoints)
df.head()

In [ ]:
fact_table_cols  = ['_id', 'borough', 'cuisine', 'name', 'grades', 'address.zipcode', 
         'address.coord'
     ]
fact_table_df = df[fact_table_cols]

In [ ]:
rating_cols = ['_id','name', 'grades'
              ]
rating_df = df[rating_cols]

In [ ]:
# Importing CSV File

In [ ]:
url = 'https://raw.githubusercontent.com/hnl2vnv/DS-2002-Project/main/fruitvegprices-2017_2022.csv'
fvprices = pd.read_csv(url,index_col=0)

In [ ]:
fvitem = ['apples', 'cabbage','onion', 'carrots','lettuce','brussels_sprouts', 'cucumbers,','pears'
             ]
fvprices_df =  fvprices[fvprices['item'].isin(fvitem)]

In [ ]:
### Creating SQL Database
#!pip install PyMySQL
#!pip install mysql-connector-python
#!pip install sqlalchemy
import os
import pymysql
import mysql.connector
from getpass import getpass
from mysql.connector import connect, Error
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [ ]:
host_name = "127.0.0.1"
user_id = "ds2002"
pwd = "UVA!1819"
db_name = "restaurants1" 

In [ ]:
try:
    with connect(
        host=host_name,
        user=user_id,
        password=pwd,
    ) as connection:
        create_db_query = "CREATE DATABASE restaurants1"
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)

In [ ]:
try:
    with connect(
        host=host_name,
        user=user_id,
        password=pwd,
        database="restaurants1",
    ) as connection:
        print(connection)
except Error as e:
    print(e)
    

In [ ]:
create_dim_date = """
CREATE TABLE IF NOT EXISTS dimension_date(
 date_key int NOT NULL,
 full_date date NULL,
 date_name char(11) NOT NULL,
 date_name_us char(11) NOT NULL,
 date_name_eu char(11) NOT NULL,
 day_of_week tinyint NOT NULL,
 day_name_of_week char(10) NOT NULL,
 day_of_month tinyint NOT NULL,
 day_of_year smallint NOT NULL,
 weekday_weekend char(10) NOT NULL,
 week_of_year tinyint NOT NULL,
 month_name char(10) NOT NULL,
 month_of_year tinyint NOT NULL,
 is_last_day_of_month char(1) NOT NULL,
 calendar_quarter tinyint NOT NULL,
 calendar_year smallint NOT NULL,
 calendar_year_month char(10) NOT NULL,
 calendar_year_qtr char(10) NOT NULL,
 fiscal_month_of_year tinyint NOT NULL,
 fiscal_quarter tinyint NOT NULL,
 fiscal_year int NOT NULL,
 fiscal_year_month char(10) NOT NULL,
 fiscal_year_qtr char(10) NOT NULL,
  PRIMARY KEY (`date_key`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
"""

conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute(create_dim_date)
    cursor.close()
    
except Error as e:
    print (e)
conn.close()

In [ ]:
create_rating_table_query = """
CREATE TABLE IF NOT EXISTS ratings (
  _id INT PRIMARY KEY,
  name VARCHAR(255),
  grades VARCHAR(255)
)
"""
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute(create_rating_table_query)
    cursor.close()
    
except Error as e:
    print (e)
conn.close()

In [ ]:
create_fvitem_table_query = """ 
CREATE TABLE IF NOT EXISTS item (
  apples VARCHAR(255),
  cabbage VARCHAR(255),
  onion   VARCHAR(255),
  carrots VARCHAR(255),
  lettuce VARCHAR(255),
  brussels_sprouts VARCHAR(255),
  cucumbers VARCHAR(255),
  pears VARCHAR(255)
)
 """
    
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute(create_fvitem_table_query)
    cursor.close()
    
except Error as e:
    print (e)

In [ ]:
create_restaurants_table_query = """
CREATE TABLE IF NOT EXISTS restaurants (
  _id INT PRIMARY KEY,
  borough VARCHAR(255),
  cuisine VARCHAR(255),
  listing_name VARCHAR(255),
  grades ,
  address_zipcode VARCHAR(255),
  address_coord VARCHAR(255),
  FOREIGN KEY (_id) REFERENCES ratings (_id)
)
"""

conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute(create_restaurants_table_query)
    cursor.close()
    
except Error as e:
    print (e)
    
conn.close()

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
# create cursor
cursor=conn.cursor()


# print(rating_df)
# print(str(i) for i in rating_df.columns.tolist())

# creating column list for insertion
cols = ",".join([str(i) for i in rating_df.columns])
print(type(cols))

# Insert DataFrame recrds one by one.
for i,row in rating_df.iterrows():
    print(row)
    sql = "INSERT INTO ratings (" +cols + ") VALUES (row)"
    cursor.execute(sql, tuple(row))

    conn.commit()

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
# create cursor
cursor=conn.cursor()

# creating column list for insertion
cols = ",".join([str(i) for i in fvprices_df.columns.tolist()])

# Insert DataFrame records one by one.
for i,row in fvprices_df.iterrows():
    sql = "INSERT INTO fvitem (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    conn.commit()

In [ ]:
checks = """
SET GLOBAL FOREIGN_KEY_CHECKS=0
"""

conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute(checks)
    cursor.close()
    
except Error as e:
    print (e)
conn.close()

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
# create cursor
cursor=conn.cursor()

# creating column list for insertion
cols = ",".join([str(i) for i in fact_table_df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in fact_table_df.iterrows():
    sql = "INSERT INTO fact_table_cols (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s) ON DUPLICATE KEY UPDATE id=id" 
    cursor.execute(sql, tuple(row))

    conn.commit()
    